In [1]:
import os
import io
import requests

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

PROJ_PATH = '/home/reese56/w266_final/'
VOTEVIEW = os.path.join(PROJ_PATH, 'data/voteview/')
VOTEVIEW_RAW = os.path.join(VOTEVIEW, 'data/voteview/raw/')
SAVE_PATH = os.path.join(VOTEVIEW, 'gen/')

In [2]:
# import voteview congressional vote data
try:
    congress_votes = pd.read_csv(os.path.join(VOTEVIEW, 'raw/congress_votes.csv'))
except FileNotFoundError:
    url= 'https://voteview.com/static/data/out/rollcalls/HSall_rollcalls.csv'
    r = requests.get(url).content
    congress_votes = pd.read_csv(io.StringIO(r.decode('utf-8')))

# import voteview members votes data
try:
    member_votes = pd.read_csv(os.path.join(VOTEVIEW, 'raw/member_votes.csv'))
except FileNotFoundError:
    url= 'https://voteview.com/static/data/out/votes/HSall_votes.csv'
    r = requests.get(url).content
    member_votes = pd.read_csv(io.StringIO(r.decode('utf-8')))

/home/reese56/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# select desired congressional sessions
cv_period = congress_votes['congress'] >= 75
cv_period = cv_period & (congress_votes['congress'] <= 114)
mv_period = member_votes['congress'] >= 75
mv_period = mv_period & (member_votes['congress'] <= 114)


# select chamber
chambers = ['House', 'Senate']
cv_chambers = congress_votes['chamber'].isin(chambers)
mv_chambers = member_votes['chamber'].isin(chambers)


# selected attributes
cv_attributes = ['congress', 'chamber', 'rollnumber','yea_count', 'nay_count','vote_result','bill_number']
mv_attributes = ['congress', 'chamber', 'rollnumber', 'icpsr', 'cast_code']

# apply selections
congress_votes = congress_votes[cv_period & cv_chambers].filter(cv_attributes, axis = 1)
member_votes = member_votes[mv_period & mv_chambers].filter(mv_attributes, axis = 1)

In [4]:
# import party majority data
majority = pd.read_csv(os.path.join(VOTEVIEW, 'gen/majority.csv'))
majority = majority[['congress','chamber','majority_gen', 'dem_count','gop_count']]

In [5]:
# creating simple majority vote categorical from yea nay vote counts
congress_votes['status'] = np.where(congress_votes['yea_count'] > congress_votes['nay_count'], 'passed','failed')
congress_votes['status'] = np.where(congress_votes['yea_count'] == congress_votes['nay_count'],
                                    'tie_vote',congress_votes['status'])

In [6]:
# examinig the proportion of votes designated as a bill in all sessions for both chambers combined
bills = ~congress_votes['bill_number'].isna()
passed = congress_votes['status'] == 'passed'
tied = congress_votes['status'] == 'tie_vote'

prop_passed = (bills & passed).sum()/congress_votes.shape[0]
print('proportion of bills passed:', prop_passed)
prop_tied = (bills & tied).sum()/congress_votes.shape[0]
print('proportion of bills ties:', prop_tied)

proportion of bills passed: 0.564588419633422
proportion of bills ties: 0.0033159639981051635


In [7]:
# creating simple yea, nay, abstained vote cast categorical
member_votes['cast'] = np.where(member_votes['cast_code'].isin([1,2,3]), 'yea','present')
member_votes['cast'] = np.where(member_votes['cast_code'].isin([4,5,6]), 'nay', member_votes['cast'])
member_votes['cast'] = np.where(member_votes['cast_code'].isin([7,8,9]), 'abstained', member_votes['cast'])
member_votes['cast'].value_counts()

yea          9496501
nay          5171912
abstained    1174761
Name: cast, dtype: int64

In [8]:
votes_p = member_votes.merge(congress_votes, how = 'inner', on = ['congress','chamber','rollnumber'])

# checking megre
print('member_votes:',member_votes.shape[0])
print('congress_votes:',congress_votes.shape[0])
print('votes_p:',votes_p.shape[0])

member_votes: 15843174
congress_votes: 54886
votes_p: 15843174


In [9]:
# filter for bills
votes_p = votes_p[~votes_p['bill_number'].isna()]

In [10]:
# temporary chamber dataframes
house_bool = votes_p['chamber'] == 'House'
house_votes = votes_p[house_bool]
senate_votes = votes_p[~house_bool]

# passed in house
hp = house_votes['bill_number'].isin(senate_votes['bill_number']).sum()/house_votes.shape[0]

# passed in senate
sp = senate_votes['bill_number'].isin(house_votes['bill_number']).sum()/senate_votes.shape[0]

print('Proportion votes in House:',hp)
print('Proportion votes in Senate:',sp)

Proportion votes in House: 0.36786018321672265
Proportion votes in Senate: 0.6341761038110686


In [11]:
# import voteview congressional members dw-nominate score data
try:
    member_ideology = pd.read_csv(os.path.join(VOTEVIEW_RAW, 'raw/member_ideology.csv'))
except FileNotFoundError:
    url= 'https://voteview.com/static/data/out/members/HSall_members.csv'
    r = requests.get(url).content
    member_ideology = pd.read_csv(io.StringIO(r.decode('utf-8')))

In [12]:
# select major political parties
party_codes = member_ideology['party_code'].isin([100,200])

# select desired congressional sessions
period = member_ideology['congress'] >= 75
period = period & (member_ideology['congress'] <= 114)

# select desired attributes
attributes = ['congress','chamber','party_code','icpsr']

# select chambers
chambers = member_ideology['chamber'].isin(['House','Senate'])

# apply selections
member_party = member_ideology[party_codes & period & chambers].filter(attributes)

In [13]:
# merge member party information with congressional majority information
member_majority = member_party.merge(majority, how = 'inner', on = ['congress','chamber'])

# post merge examination
print('member_party:',member_party.shape[0])
print('majority:',majority.shape[0])
print('member_majority:',member_majority.shape[0])

member_party: 21806
majority: 80
member_majority: 21806


In [14]:
# combining member party and party count information with member and congressional votes taken
combined_votes = votes_p.merge(member_majority, how = 'inner', on = ['congress','chamber','icpsr'])
combined_votes.drop(['yea_count','nay_count','vote_result','cast_code'], axis = 1, inplace = True)

# merege examination
print('votes_p:',votes_p.shape[0])
print('member_majority:',member_majority.shape[0])
print('combined_votes:',combined_votes.shape[0])
print('entries lost:', votes_p.shape[0] - combined_votes.shape[0])

votes_p: 13775137
member_majority: 21806
combined_votes: 13699203
entries lost: 75934


In [15]:
# missing congressional sessions from member_majority data frame
len(votes_p[~votes_p['congress'].isin(member_majority['congress'].values)]['congress'].unique())

0

In [16]:
# missing congressional sessions from vote_p data frame
member_majority[~member_majority['congress'].isin(votes_p['congress'].values)]['congress'].unique()

array([ 99, 100])

In [17]:
# count of members missing from vote_p dataframe
len(member_majority[~member_majority['icpsr'].isin(votes_p['icpsr'].values)]['icpsr'].unique())

34

In [18]:
# count of members missing from member_majority dataframe
len(votes_p[~votes_p['icpsr'].isin(member_majority['icpsr'].values)]['icpsr'].unique())

35

So far as I can tell, the entries lost here come from a missing the 99th and 100th sessions, and 34 members in the `votes_p` data frame. The casuse has yet to be identified.

In [19]:
combined_votes.to_csv(os.path.join(SAVE_PATH, 'combined_votes.csv'), index = False)